In [ ]:
pip install langgraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.


In [ ]:
import os
from typing import TypedDict,Annotated,List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, image

Define Agent
### bold text

In [ ]:
pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

class plannerState(TypedDict):
  messages : Annotated[List[HumanMessage | AIMessage],"the messages in the conversation"]
  city : str
  interests: List[str]
  itinerary: str

In [ ]:
#Define the llm
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("What is Multi AI Agent")
result.content

'A Multi-Agent System (MAS) is a system composed of multiple autonomous agents that interact with each other and their environment to achieve common or individual goals. In the context of Artificial Intelligence (AI), a Multi-AI Agent refers to a system where multiple AI agents collaborate, coordinate, or compete with each other to solve complex problems or achieve specific objectives.\n\nCharacteristics of Multi-AI Agent Systems:\n\n1. **Autonomy**: Each AI agent operates independently, making decisions based on its own goals, preferences, and knowledge.\n2. **Interactions**: AI agents interact with each other through communication, cooperation, or competition to achieve their objectives.\n3. **Distributed Problem-Solving**: Multiple AI agents work together to solve complex problems that are difficult or impossible for a single agent to solve.\n4. **Decentralized Control**: There is no centralized control or single point of decision-making; instead, decision-making is distributed amon

In [ ]:
pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful travel assistant.Create a day trip itinerary for {city} based on the user's interests: {interests}.Provide a brief, itinerary."),("human","Create an bullated itinerary for my day trip"),])


Define agent function



In [ ]:
def input_city(state: plannerState) -> plannerState:
  print("Please enter the city you want to visit for your day trip: ")
  user_message = input("Your Input:")
  return{
      **state,
      "city": user_message,
      "messages": state['messages'] + [HumanMessage(content=user_message)]

  }
def input_interest(state: plannerState) -> plannerState:
  print(f"Please enter your interests for the trip to: {state['city']} (comma-seperated) ")
  user_message = input("Your Input:")
  return{
      **state,
      "interests": [interest.strip() for interest in user_message.split(",")],
      "messages": state['messages'] + [HumanMessage(content=user_message)]
  }
def create_itinerary(state: plannerState) -> plannerState:
    print(f"Creating an itinerary for {state['city']} based on interests : {','.join(state['interests'])}")
    response =llm.invoke(itinerary_prompt.format_messages(city = state['city'], interests = ','.join(state['interests'])))
    print("\nFinal Itinerary: ")
    print(response.content)
    return{
      **state,
      "messages": state['messages'] + [AIMessage(content=response.content)],
      "itinerary":response.content
  }
#Define the Gradio applications
def travel_planner(city: str, interests: str):
  #Initializing state
  state={
      "messages":[],
      "city":"",
      "interests":[],
      "itinerary":"",
  }
  #Process the city and interests inputs
  state = input_city(state)
  state = input_interest(state)
  #Generate the itinerary
  state = create_itinerary(state)

  return state['itinerary']

#Build the Gradio interface
import gradio as gr

interface = gr.Interface(
    fn=travel_planner,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=[
        gr.Textbox(label="Enter the city for your day trip"),
        gr.Textbox(label="Enter your interests (comma-seperated)"),
    ],
    outputs=gr.Textbox(label="Generated Itinerary"),
    title="Travel Itinerary planner",
    description="Enter a city and your interests to generate a personalized day trip itinerary."

)
#Launch the Gradio application
interface.launch()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


theme_schema%400.0.1.json: 0.00B [00:00, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11dc74453198f27d52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
